In [262]:
import fastText
import math
import linecache
import numpy as np 
from numpy import random
from random import sample
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint
from keras.layers import *
from keras import *
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
import re
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

# from attention_utils import get_activations, get_data_recurrent


In [263]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [264]:
ft = fastText.load_model("/home/jindal/notebooks/fastText/wiki.de.bin")

nb_embedding_dims = ft.get_dimension()
nb_sequence_length = 75

In [265]:
def twitter_tokenizer(textline):
    textLine = re.sub(r'http\S+', 'URL', textline)
    textline = re.sub('@[\w_]+', 'USER_MENTION', textline)
    textline = re.sub('\|LBR\|', '', textline)
    textline = re.sub('\.\.\.+', '...', textline)
    textline = re.sub('!!+', '!!', textline)
    textline = re.sub('\?\?+', '??', textline)
    words = re.compile('[\U00010000-\U0010ffff]|[\w-]+|[^ \w\U00010000-\U0010ffff]+', re.UNICODE).findall(textline.strip())
    words = [w.strip() for w in words if w.strip() != '']
    # print(words)
    return(words)

In [266]:
word_vectors_ft = {}
def process_features(textline, nb_sequence_length, nb_embedding_dims):
    words = twitter_tokenizer(textline)
    features_ft = np.zeros((nb_sequence_length, nb_embedding_dims))
    features_idx = np.zeros(nb_sequence_length)
    max_words = min(len(words), nb_sequence_length)
    idx = nb_sequence_length - len(words[:max_words])
    for w in words[:max_words]:
        if w in word_vectors_ft:
            wv = word_vectors_ft[w]
        else:
            wv = ft.get_word_vector(w.lower())
            word_vectors_ft[w] = wv
        features_ft[idx] = wv
        
        idx = idx + 1
    return features_ft

In [267]:
def sequential_generator(filename, batch_size, check:
                         'to check if all lines in file are of same length.To check enter the len of line after splitting it by tabs' = None):
    
    f = open(filename)
    while True:
        batch_features_ft = np.zeros((batch_size, nb_sequence_length, nb_embedding_dims))
        batch_labels = np.zeros((batch_size, 2))
        for i in range(batch_size):
            line = f.readline()
            if ("" == line):
                f.seek(0)
                line = f.readline()
            data = line.strip().split('\t')
            if check:
                if len(data)!=check:
                    i-=1
                    continue
            batch_features_ft[i] = process_features(data[0], nb_sequence_length, nb_embedding_dims)
            batch_labels[i] = to_categorical(0 if data[1] == 'OTHER' else 1, n_labels)
            
        yield ([batch_features_ft], batch_labels)

In [268]:
def train_dev_sentences(filetrain,filedev, check:'to check if lines of file are all same lenght after separating by tab'):
    train_lines = [line.strip().split("\t") for line in open(filetrain) if len(line.strip().split('\t'))==check]
    dev_lines = [line.strip().split("\t") for line in open(filedev) if len(line.strip().split('\t'))==check]

    train_sentences = [x[0] for x in train_lines]
    train_labels = [0 if x[1] == "OTHER" else 1 for x in train_lines]

    dev_sentences = [x[0] for x in dev_lines]
    dev_labels = [0 if x[1] == "OTHER" else 1 for x in dev_lines]

    return (train_sentences, train_labels, dev_sentences, dev_labels)


In [269]:
train_sentences, train_labels, dev_sentences, dev_labels = train_dev_sentences(filetrain='/home/jindal/notebooks/jindal/NER/language_model/FB_train.csv',
                   filedev='/home/jindal/notebooks/jindal/NER/language_model/FB_dev.csv', check=2)

In [270]:
print(len(dev_sentences))

774


In [271]:
dev_sentences[0]

'Oh, etwa mal wieder Verbindungen nach Deutschland? Salafisten und sonstigem kriminellem und islam-extremistischem Mist, macht man es hier ja auch besonders gemütlich. Dürfte niemanden wirklich wundern.'

In [278]:
print(len(devLabels))

774


In [272]:
n_labels =2

In [273]:
def compile_model():
    model_input_embedding = Input(shape = (nb_sequence_length, nb_embedding_dims))
    lstm_block = Bidirectional(LSTM(100, dropout = 0.5, return_sequences=True))(model_input_embedding)
    lstm_block = LeakyReLU()(lstm_block)

    filter_sizes = (3, 4, 5)
    conv_blocks = []
    for sz in filter_sizes:
        conv = Conv1D(
            filters = 200,
            kernel_size = sz,
            padding = 'valid',
            strides = 1
        )(lstm_block)
        conv = LeakyReLU()(conv)
        conv = GlobalMaxPooling1D()(conv)
        conv = Dropout(0.5)(conv)
        conv_blocks.append(conv)
    model_concatenated = concatenate([conv_blocks[0], conv_blocks[1], conv_blocks[2]])
    # model_concatenated = Dropout(0.8)(model_concatenated)
    model_concatenated = Dense(100)(model_concatenated)
    model_concatenated = LeakyReLU()(model_concatenated)
    model_output = Dense(n_labels, activation = "softmax")(model_concatenated)
    new_model = Model(model_input_embedding, model_output)
    new_model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])
#     new_model.summary()
    return new_model

In [274]:
print(dev_sentences[1])

Die sollen sich besser mal um die Antifa & linksextreme Faschisten kümmern anstatt sich mit harmlosen Flyern zu beschäftigen.


In [306]:
def test_model(generator, train_sentences, devLabels, number_of_tests,number_of_epochs, filename_to_log, 
               filename_to_save_weigths, batch_size, train_file:'filepath for traininig',
               f1_measure:'binary/macro etc', pos_label:'only if binary f1',
               load_model_weights=False,model_weights_file:'give filepath as str'=None, 
               nb_sequence_length = nb_sequence_length, nb_embedding_dims= nb_embedding_dims, check_for_generator=None ):
    
    f = open(filename_to_log,"w")
    
    max_f1=0
    max_p=0
    max_r=0
    max_a=0
    total_f1=0
    total_prec=0
    total_acc=0
    total_recall=0
    
    for test_number in range(number_of_tests):
        print("Test %d/%d" %(test_number+1, number_of_tests))
        model = compile_model()

        # transfer learning
        if load_model_weights and model_weights_file:
                model.load_weights(model_weights_file)


        epochs = number_of_epochs
        batch_size = batch_size
        steps_per_epoch = math.ceil(samples_per_epoch / batch_size)
        # checkpoint = ModelCheckpoint('best_classification_model_million_post_corpus.h5', monitor='val_acc',save_best_only = True, 
    #                                  save_weights_only = True)

        for epoch in range(epochs):
            print("Epoch: %d" %(epoch+1))
            model.fit_generator(
                generator(train_file, batch_size, check_for_generator), 
                steps_per_epoch=steps_per_epoch, epochs=1,
        #         validation_data = sequential_generator('/home/jindal/notebooks/jindal/NER/language_model/million_post_corpus_dev.csv', batch_size),
        #         validation_steps = math.ceil(len(dev_sentences) / batch_size),
        #         callbacks = [checkpoint]
            )

            testset_features = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding_dims))
            for i in range(len(dev_sentences)):
                testset_features[i] = process_features(dev_sentences[i], nb_sequence_length, nb_embedding_dims)
            results = model.predict(testset_features)


            idx2Label = {0 : "OTHER", 1 : "OFFENSIVE"}
            predLabels = results.argmax(axis=-1)
            devLabels = devLabels
            f1 = f1_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label) # offensive is the major class. So other is minor
            r = recall_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
            p = precision_score(devLabels, predLabels, average=f1_measure, pos_label=pos_label)
            a = accuracy_score(devLabels, predLabels)
            if max_f1 < f1:
                print("model saved. F1 is %f" %(f1))
                model.save(filename_to_save_weigths)
                max_f1 = f1
                max_p = p
                max_r = r
                max_a = a
            text = "prec: "+ str(p)+" rec: "+str(r) +" f1: "+str(f1) +" acc: "+str(a)+" \n"
            print("Test-Data: Prec: %.3f, Rec: %.3f, F1: %.3f, Acc: %.3f" % (p, r, f1, a))
        to_write= "prec: "+ str(max_p)+" rec: "+str(max_r) +" f1: "+str(max_f1) +" acc: "+str(max_a)+" \n"
        print(to_write)
        f.write(to_write)
        total_f1+=max_f1
        total_prec+=max_p
        total_acc+=max_a
        total_recall+=max_r    
        print("*****************************************************************************")
    final_text = "avg_prec: " +str(total_prec/number_of_tests)+" total_rec: "+str(total_recall/number_of_tests) +" total_f1: "+str(total_f1/number_of_tests) +" total_acc: "+str(total_acc/number_of_tests)+" \n"
    print(final_text)
    f.write(final_text)
    f.close()




In [304]:
# total_f1

print(max_f1)

0.8


In [286]:
f = open('results_pretraining_FB.txt','a+')
final_text = "avg_prec: " +str(total_prec/number_of_tests)+" total_rec: "+str(total_recall/number_of_tests) +" total_f1: "+str(total_f1/number_of_tests) +" total_acc: "+str(total_acc/number_of_tests)+" \n"
f.write(final_text)
f.close()

In [283]:
generator = sequential_generator
train_sentences = train_sentences
devLabels = dev_labels
number_of_tests = 3
number_of_epochs = 50
fb_pretraining_log = '/home/jindal/notebooks/jindal/NER/language_model/results_pretraining_FB.txt' 
fb_pretraining_save_weigths='/home/jindal/notebooks/jindal/NER/language_model/model_pretrained_FB.h5'
batch_size=32
fb_train_file='/home/jindal/notebooks/jindal/NER/language_model/FB_train.csv'
f1_measure='binary'
pos_label=0
load_model_weights=False
# model_weights_file:'give filepath as str'=None, 
nb_sequence_length = nb_sequence_length
nb_embedding_dims= nb_embedding_dims
check_for_generator=2



In [284]:
test_model(generator=generator, train_sentences=train_sentences, devLabels=devLabels, number_of_tests= number_of_tests,
          number_of_epochs=number_of_epochs, filename_to_log=fb_pretraining_log, filename_to_save_weigths=fb_pretraining_save_weigths,
          batch_size=batch_size,train_file=fb_train_file, f1_measure=f1_measure, pos_label=pos_label, load_model_weights=load_model_weights,
          nb_sequence_length=nb_sequence_length, nb_embedding_dims=nb_embedding_dims, check_for_generator= check_for_generator)

Test 1/3
Epoch: 1
Epoch 1/1
132/132 [==============================] - 35s 268ms/step - loss: 0.5860 - acc: 0.7095
model saved. F1 is 0.183333
Test-Data: Prec: 0.647, Rec: 0.107, F1: 0.183, Acc: 0.747
Epoch: 2
Epoch 1/1
132/132 [==============================] - 31s 233ms/step - loss: 0.5403 - acc: 0.7391
model saved. F1 is 0.327526
Test-Data: Prec: 0.580, Rec: 0.228, F1: 0.328, Acc: 0.751
Epoch: 3
Epoch 1/1
132/132 [==============================] - 29s 222ms/step - loss: 0.5130 - acc: 0.7640
model saved. F1 is 0.468571
Test-Data: Prec: 0.569, Rec: 0.398, F1: 0.469, Acc: 0.760
Epoch: 4
Epoch 1/1
132/132 [==============================] - 29s 219ms/step - loss: 0.4788 - acc: 0.7751
Test-Data: Prec: 0.609, Rec: 0.379, F1: 0.467, Acc: 0.770
Epoch: 5
Epoch 1/1
132/132 [==============================] - 30s 228ms/step - loss: 0.4546 - acc: 0.7886
model saved. F1 is 0.509695
Test-Data: Prec: 0.594, Rec: 0.447, F1: 0.510, Acc: 0.771
Epoch: 6
Epoch 1/1
132/132 [==============================]

132/132 [==============================] - 30s 226ms/step - loss: 0.0306 - acc: 0.9889
Test-Data: Prec: 0.638, Rec: 0.607, F1: 0.622, Acc: 0.804
Epoch: 50
Epoch 1/1
132/132 [==============================] - 30s 228ms/step - loss: 0.0358 - acc: 0.9870
Test-Data: Prec: 0.685, Rec: 0.558, F1: 0.615, Acc: 0.814
*****************************************************************************
Test 2/3
Epoch: 1
Epoch 1/1
132/132 [==============================] - 35s 268ms/step - loss: 0.5835 - acc: 0.7119
Test-Data: Prec: 0.564, Rec: 0.107, F1: 0.180, Acc: 0.740
Epoch: 2
Epoch 1/1
132/132 [==============================] - 30s 227ms/step - loss: 0.5442 - acc: 0.7356
Test-Data: Prec: 0.613, Rec: 0.238, F1: 0.343, Acc: 0.757
Epoch: 3
Epoch 1/1
132/132 [==============================] - 30s 224ms/step - loss: 0.5066 - acc: 0.7597
Test-Data: Prec: 0.650, Rec: 0.325, F1: 0.434, Acc: 0.774
Epoch: 4
Epoch 1/1
132/132 [==============================] - 30s 224ms/step - loss: 0.4751 - acc: 0.7756
Test-

132/132 [==============================] - 29s 217ms/step - loss: 0.0297 - acc: 0.9896
Test-Data: Prec: 0.637, Rec: 0.621, F1: 0.629, Acc: 0.805
Epoch: 49
Epoch 1/1
132/132 [==============================] - 29s 217ms/step - loss: 0.0449 - acc: 0.9858
Test-Data: Prec: 0.625, Rec: 0.583, F1: 0.603, Acc: 0.796
Epoch: 50
Epoch 1/1
132/132 [==============================] - 29s 218ms/step - loss: 0.0341 - acc: 0.9891
Test-Data: Prec: 0.631, Rec: 0.597, F1: 0.613, Acc: 0.800
*****************************************************************************
Test 3/3
Epoch: 1
Epoch 1/1
132/132 [==============================] - 35s 266ms/step - loss: 0.5850 - acc: 0.7131
Test-Data: Prec: 0.618, Rec: 0.102, F1: 0.175, Acc: 0.744
Epoch: 2
Epoch 1/1
132/132 [==============================] - 28s 211ms/step - loss: 0.5407 - acc: 0.7415
Test-Data: Prec: 0.652, Rec: 0.146, F1: 0.238, Acc: 0.752
Epoch: 3
Epoch 1/1
132/132 [==============================] - 28s 214ms/step - loss: 0.5091 - acc: 0.7659
Test

132/132 [==============================] - 29s 220ms/step - loss: 0.0384 - acc: 0.9856
Test-Data: Prec: 0.619, Rec: 0.631, F1: 0.625, Acc: 0.798
Epoch: 48
Epoch 1/1
132/132 [==============================] - 29s 219ms/step - loss: 0.0306 - acc: 0.9872
Test-Data: Prec: 0.654, Rec: 0.587, F1: 0.619, Acc: 0.807
Epoch: 49
Epoch 1/1
132/132 [==============================] - 29s 218ms/step - loss: 0.0306 - acc: 0.9867
Test-Data: Prec: 0.642, Rec: 0.680, F1: 0.660, Acc: 0.814
Epoch: 50
Epoch 1/1
132/132 [==============================] - 29s 223ms/step - loss: 0.0415 - acc: 0.9825
Test-Data: Prec: 0.667, Rec: 0.592, F1: 0.627, Acc: 0.813
*****************************************************************************


TypeError: bad operand type for unary +: 'str'

# TRANSFER LEARNING HERE


In [294]:
n_labels =2

In [295]:
train_sentences, train_labels, dev_sentences, dev_labels = train_dev_sentences(filetrain='/home/gwiedemann/notebooks/OffLang/sample_train.txt',
                   filedev='/home/gwiedemann/notebooks/OffLang/sample_dev.txt', check=3)

In [298]:
print(dev_sentences[0])
print(dev_labels[:20])

@FilmElf_yt Diese ganzen leute die du angesprochen hast sind untermenschen ja aber das alte Deutschland und deutsche volk waren ehrenwerte leute und ein stolzes Land
[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1]


In [307]:
generator = sequential_generator
train_sentences = train_sentences
devLabels = dev_labels
number_of_tests = 5
number_of_epochs = 50
fb_tl_log = '/home/jindal/notebooks/jindal/NER/language_model/results_transfer_learning_fb.txt' 
fb_tl_save_weigths='/home/jindal/notebooks/jindal/NER/language_model/classification_model_tl_FB.h5'
batch_size=32
fb_tl_train_file='/home/gwiedemann/notebooks/OffLang/sample_train.txt'
f1_measure='binary'
pos_label=1
load_model_weights=True
model_weights_file = '/home/jindal/notebooks/jindal/NER/language_model/model_pretrained_FB.h5'
nb_sequence_length = nb_sequence_length
nb_embedding_dims= nb_embedding_dims
check_for_generator=3

In [308]:
test_model(generator=generator, train_sentences=train_sentences, devLabels=devLabels, number_of_tests= number_of_tests,
          number_of_epochs=number_of_epochs, filename_to_log=fb_tl_log, filename_to_save_weigths=fb_tl_save_weigths,
          batch_size=batch_size,train_file=fb_tl_train_file, f1_measure=f1_measure, pos_label=pos_label, load_model_weights=load_model_weights,
          model_weights_file = model_weights_file, nb_sequence_length=nb_sequence_length, nb_embedding_dims=nb_embedding_dims, check_for_generator= check_for_generator)

Test 1/5
Epoch: 1
Epoch 1/1
132/132 [==============================] - 35s 268ms/step - loss: 0.6149 - acc: 0.7180
model saved. F1 is 0.565310
Test-Data: Prec: 0.660, Rec: 0.494, F1: 0.565, Acc: 0.749
Epoch: 2
Epoch 1/1
132/132 [==============================] - 30s 224ms/step - loss: 0.4815 - acc: 0.7704
model saved. F1 is 0.650647
Test-Data: Prec: 0.642, Rec: 0.659, F1: 0.651, Acc: 0.766
Epoch: 3
Epoch 1/1
132/132 [==============================] - 29s 223ms/step - loss: 0.4278 - acc: 0.8033
Test-Data: Prec: 0.669, Rec: 0.629, F1: 0.649, Acc: 0.775
Epoch: 4
Epoch 1/1
132/132 [==============================] - 30s 224ms/step - loss: 0.3966 - acc: 0.8161
model saved. F1 is 0.683274
Test-Data: Prec: 0.651, Rec: 0.719, F1: 0.683, Acc: 0.780
Epoch: 5
Epoch 1/1
132/132 [==============================] - 29s 223ms/step - loss: 0.3651 - acc: 0.8279
Test-Data: Prec: 0.695, Rec: 0.633, F1: 0.663, Acc: 0.787
Epoch: 6
Epoch 1/1
132/132 [==============================] - 30s 226ms/step - loss: 0.

132/132 [==============================] - 29s 216ms/step - loss: 0.0420 - acc: 0.9853
Test-Data: Prec: 0.673, Rec: 0.708, F1: 0.690, Acc: 0.790
prec: 0.6688741721854304 rec: 0.7565543071161048 f1: 0.7100175746924429 acc: 0.7957920792079208 

*****************************************************************************
Test 2/5
Epoch: 1
Epoch 1/1
132/132 [==============================] - 35s 268ms/step - loss: 0.6165 - acc: 0.7195
Test-Data: Prec: 0.665, Rec: 0.513, F1: 0.579, Acc: 0.754
Epoch: 2
Epoch 1/1
132/132 [==============================] - 28s 216ms/step - loss: 0.4805 - acc: 0.7689
Test-Data: Prec: 0.681, Rec: 0.584, F1: 0.629, Acc: 0.772
Epoch: 3
Epoch 1/1
132/132 [==============================] - 28s 214ms/step - loss: 0.4458 - acc: 0.7914
Test-Data: Prec: 0.672, Rec: 0.622, F1: 0.646, Acc: 0.775
Epoch: 4
Epoch 1/1
132/132 [==============================] - 28s 215ms/step - loss: 0.4089 - acc: 0.8146
Test-Data: Prec: 0.673, Rec: 0.625, F1: 0.649, Acc: 0.776
Epoch: 5
Epoch

132/132 [==============================] - 28s 210ms/step - loss: 0.0317 - acc: 0.9875
Test-Data: Prec: 0.648, Rec: 0.723, F1: 0.683, Acc: 0.778
Epoch: 50
Epoch 1/1
132/132 [==============================] - 29s 221ms/step - loss: 0.0382 - acc: 0.9860
Test-Data: Prec: 0.695, Rec: 0.700, F1: 0.698, Acc: 0.800
prec: 0.6688741721854304 rec: 0.7565543071161048 f1: 0.7100175746924429 acc: 0.7957920792079208 

*****************************************************************************
Test 3/5
Epoch: 1
Epoch 1/1
132/132 [==============================] - 36s 276ms/step - loss: 0.6262 - acc: 0.7121
Test-Data: Prec: 0.626, Rec: 0.532, F1: 0.575, Acc: 0.740
Epoch: 2
Epoch 1/1
132/132 [==============================] - 28s 215ms/step - loss: 0.4884 - acc: 0.7694
Test-Data: Prec: 0.665, Rec: 0.588, F1: 0.624, Acc: 0.766
Epoch: 3
Epoch 1/1
132/132 [==============================] - 29s 218ms/step - loss: 0.4409 - acc: 0.7940
Test-Data: Prec: 0.672, Rec: 0.629, F1: 0.650, Acc: 0.776
Epoch: 4
Epoc

132/132 [==============================] - 32s 240ms/step - loss: 0.0397 - acc: 0.9846
Test-Data: Prec: 0.704, Rec: 0.678, F1: 0.691, Acc: 0.800
Epoch: 48
Epoch 1/1
132/132 [==============================] - 31s 238ms/step - loss: 0.0359 - acc: 0.9858
Test-Data: Prec: 0.681, Rec: 0.719, F1: 0.699, Acc: 0.796
Epoch: 49
Epoch 1/1
132/132 [==============================] - 32s 241ms/step - loss: 0.0226 - acc: 0.9912
Test-Data: Prec: 0.687, Rec: 0.689, F1: 0.688, Acc: 0.793
Epoch: 50
Epoch 1/1
132/132 [==============================] - 31s 236ms/step - loss: 0.0302 - acc: 0.9905
Test-Data: Prec: 0.677, Rec: 0.715, F1: 0.696, Acc: 0.793
prec: 0.6710097719869706 rec: 0.7715355805243446 f1: 0.7177700348432056 acc: 0.7995049504950495 

*****************************************************************************
Test 4/5
Epoch: 1
Epoch 1/1
132/132 [==============================] - 41s 309ms/step - loss: 0.6243 - acc: 0.7164
Test-Data: Prec: 0.649, Rec: 0.472, F1: 0.547, Acc: 0.741
Epoch: 2
Ep

132/132 [==============================] - 33s 251ms/step - loss: 0.0336 - acc: 0.9870
Test-Data: Prec: 0.668, Rec: 0.648, F1: 0.658, Acc: 0.777
Epoch: 47
Epoch 1/1
132/132 [==============================] - 32s 240ms/step - loss: 0.0402 - acc: 0.9863
Test-Data: Prec: 0.695, Rec: 0.708, F1: 0.701, Acc: 0.801
Epoch: 48
Epoch 1/1
132/132 [==============================] - 31s 236ms/step - loss: 0.0273 - acc: 0.9903
Test-Data: Prec: 0.695, Rec: 0.674, F1: 0.684, Acc: 0.795
Epoch: 49
Epoch 1/1
132/132 [==============================] - 29s 222ms/step - loss: 0.0325 - acc: 0.9860
Test-Data: Prec: 0.679, Rec: 0.682, F1: 0.680, Acc: 0.788
Epoch: 50
Epoch 1/1
132/132 [==============================] - 30s 225ms/step - loss: 0.0403 - acc: 0.9827
Test-Data: Prec: 0.728, Rec: 0.640, F1: 0.681, Acc: 0.802
prec: 0.6710097719869706 rec: 0.7715355805243446 f1: 0.7177700348432056 acc: 0.7995049504950495 

*****************************************************************************
Test 5/5
Epoch: 1
E

132/132 [==============================] - 28s 212ms/step - loss: 0.0382 - acc: 0.9870
Test-Data: Prec: 0.714, Rec: 0.693, F1: 0.703, Acc: 0.807
Epoch: 46
Epoch 1/1
132/132 [==============================] - 28s 211ms/step - loss: 0.0335 - acc: 0.9893
Test-Data: Prec: 0.747, Rec: 0.652, F1: 0.696, Acc: 0.812
Epoch: 47
Epoch 1/1
132/132 [==============================] - 28s 210ms/step - loss: 0.0338 - acc: 0.9891
Test-Data: Prec: 0.698, Rec: 0.685, F1: 0.692, Acc: 0.798
Epoch: 48
Epoch 1/1
132/132 [==============================] - 28s 211ms/step - loss: 0.0317 - acc: 0.9891
Test-Data: Prec: 0.686, Rec: 0.712, F1: 0.699, Acc: 0.797
Epoch: 49
Epoch 1/1
132/132 [==============================] - 27s 207ms/step - loss: 0.0300 - acc: 0.9884
Test-Data: Prec: 0.709, Rec: 0.674, F1: 0.691, Acc: 0.801
Epoch: 50
Epoch 1/1
132/132 [==============================] - 27s 207ms/step - loss: 0.0323 - acc: 0.9891
Test-Data: Prec: 0.688, Rec: 0.700, F1: 0.694, Acc: 0.796
prec: 0.6710097719869706 rec: 